<a href="https://colab.research.google.com/github/waqarmm/AI-Labs/blob/master/regad_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.0 MB/s eta 0:00:00


In [2]:
pip install transformers[torch]

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments

# Load your dataset (assuming it's in a CSV file)
your_dataset = pd.read_csv('new_data_with_emotions.csv')

# Drop rows with missing labels (NaN values)
your_dataset = your_dataset.dropna(subset=['emotion'])

# Extract text data, sentiment labels, and cities
text_data = your_dataset['combined_translated'].tolist()
sentiment_labels = your_dataset['emotion'].tolist()
cities = your_dataset['city'].tolist()

# Split the dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    text_data,
    sentiment_labels,
    test_size=0.2,
    random_state=42
)

# Convert sentiment labels to numerical values
label_dict = {'Love': 0, 'Happy': 1, 'Neutral': 2, 'Sad': 3, 'Angry': 4}  # Map sentiment labels to numerical values
train_labels = [label_dict[label] for label in train_labels]
val_labels = [label_dict[label] for label in val_labels]

# Initialize GPT-2 tokenizer and encode text data
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set the EOS token as the padding token

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Rest of the code remains the same...

# Define datasets using PyTorch Dataset class
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

# Initialize GPT-2 model for sequence classification
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=len(label_dict))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=1,  # Set batch size to 1
    per_device_eval_batch_size=1,   # Set batch size to 1 for evaluation
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Start model training
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.100300,0.621905


Epoch,Training Loss,Validation Loss
1,0.100300,0.621905
2,0.000400,0.467368
3,0.000100,0.505144


TrainOutput(global_step=17148, training_loss=0.5138655846596313, metrics={'train_runtime': 3098.3271, 'train_samples_per_second': 5.535, 'train_steps_per_second': 5.535, 'total_flos': 4472086413957120.0, 'train_loss': 0.5138655846596313, 'epoch': 3.0})

In [20]:
import torch

# Assuming you've already trained the model and initialized the Trainer (trainer) and model

# Prepare a sample query
query = "I am feeling happy and excited today. I would like to visit"

# Tokenize the query
tokenized_input = tokenizer(query, return_tensors="pt")

# Check if CUDA (GPU) is available and move the input tensor to the appropriate device
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenized_input.to(device)

# Move the model to the same device as the input tensor
model.to(device)

# Make predictions
with torch.no_grad():
    outputs = model(**tokenized_input)
    logits = outputs.logits

# Get the predicted class (sentiment label)
predicted_class = torch.argmax(logits, dim=1).item()

# Reverse the label dictionary to get the sentiment label from the predicted class
reverse_label_dict = {v: k for k, v in label_dict.items()}
predicted_sentiment = reverse_label_dict[predicted_class]

# Print the predicted sentiment
print(f"The predicted sentiment based on the query is: {predicted_sentiment}")


The predicted sentiment based on the query is: Love


In [12]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

